# <b>背景</b>

目前妊娠期糖尿病（GDM）的诊断依靠糖耐量试验（OGTT），一种葡萄糖负荷试验，用以了解胰岛β细胞功能和机体对血糖的调节能力，是诊断糖尿病的确诊试验。诊断GDM的标准为空腹血糖≥5.1mmol/L，1小时≥10.0mmol/L，2小时大于等于8.5mmol/L。如果空腹血糖≥7.0mmol/L，或者2小时血糖大于≥11.1mmol/L，则诊断为糖尿病DM。

糖化血红蛋白（HbA1c）是判断糖尿病患者血糖控制良好与否非常重要的一个指标，可以反映患者近8-12周的血糖控制情况。糖化血红蛋白（HbA1c）在妊娠期没有明确的参考值范围，当HbA1c≥6.5%，则诊断为DM。对于诊断GDM则没有参考值。

# <b>任务</b>

已有109031条产科检查数据，每条记录为一个检测项目的结果，可能是OGTT、HbA1c或其他检查项目数据。而为了判断HbA1c的诊断效果，我们需要将OGTT和HbA1c匹配。

# <b>第一步 引包</b>

In [2]:
#引包：引入所需python包
import xlrd
import os
import re
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve,auc
import matplotlib as mpl
from matplotlib import pyplot as plt
from numpy import nan
from sklearn import linear_model, datasets
from sklearn.model_selection import train_test_split
import seaborn as sns
import time

from scipy.stats import kstest
from scipy import stats

import warnings
warnings.filterwarnings('ignore')

# <b>第二步 读取多个excel文件数据</b>

1）设置默认目录

2）对目录dir下每个对象进行for循环操作；用if语句是过滤掉掉子目录；

3）打开excel文件的第一表单，读取该表单的所有数据；并追加到数据框df中

4）按字段“检查日期”升序排序


In [4]:
dir = r"./data/"
print(os.listdir(dir))

['2016.xls', '2017.xls', '2018.xls']


In [33]:
df = pd.DataFrame()
for file in os.listdir(dir) :
    if not os.path.isdir(file):
        data = xlrd.open_workbook(dir+file).sheets()[0]
        df1 = pd.read_excel(dir+file,data.name,index_col=None,na_values=['9999'])
        df = df.append(df1)  
df = df.sort_values(by = "检查日期")
df.shape

(109031, 11)

# <b>第三步 预处理 </b>
数据清理/预处理是一个非常繁琐的工作，根据业务需求和数据质量而不同，没有标准的流程。下面根据实列来了解常用的一些操作：

<b>1）衍生字段</b>

从“检查日期”中提取年份信息

In [34]:
df["year"] = pd.to_datetime(df["检查日期"],format = "%Y%m%d").dt.year

<b>2）数据转换 </b>

从字符型字段“年龄”中提取整数型年龄信息“age”

 删除原来的“年龄”字段

In [35]:
df["age"] = df["年龄"].map(lambda x:x.rstrip("岁")).astype("int")
df = df.drop(["年龄"],axis=1)

<b>4）数据离散化 </b>

设置分箱阀值[0,22,30,35,42,50]

对年龄根据分箱阀值进行分段，衍生新变量“age_range”

In [36]:
bin = [0,22,30,35,42,50]
df["age_range"] = pd.cut(df["age"],bin).astype("str")
df.head(2)

,,INSPECTION_ID,登记号,检查日期,检验目的,审核时间,项目名称,结果,单位,参考值,year,age,age_range
0,1.0,20160104G0050013,20429751,20160104,75g糖耐量(空腹)+75g糖耐量（1H）+75g糖耐量（2H）,2016-01-04 13:06:25,75g一小时血糖(妊娠),5.40,mmol/L,<10.0,2016,25,"(22.0, 30.0]"
78,79.0,20160104G0050169,20457496,20160104,75g糖耐量(空腹)+75g糖耐量（1H）+75g糖耐量（2H）,2016-01-04 13:31:29,75g一小时血糖(妊娠),8.50,mmol/L,<10.0,2016,39,"(35.0, 42.0]"


<b>5) 过滤数据</b>

建一个列表values=【'75g一小时血糖(妊娠)','75g二小时血糖(妊娠)', '空腹血糖（妊娠）', '糖化血红蛋白A1c'】;

筛选所有OGTT和HbA1c检测记录，赋值给数据框df;

建一个所有做了HbA1c检测的列表list;

筛选做过HbA1c检测的人群的所有OGTT和HbA1c记录，赋值给数据框df;

筛选HbA1c检测记录，赋值给数据框HbA1c_ref(HbA1c检测记录）;

筛选df检测记录，赋值给数据框df（OGTT检测记录）


In [37]:
values = ['75g一小时血糖(妊娠)','75g二小时血糖(妊娠)', '空腹血糖（妊娠）', '糖化血红蛋白A1c'] #'75g一小时血糖','75g二小时血糖','空腹血糖']
df = df.loc[df["项目名称"].isin(values)]
list = df.loc[df["项目名称"] =='糖化血红蛋白A1c',"登记号" ]
df = df.loc[df["登记号"].isin(list)]
values = ["糖化血红蛋白A1c"]
HbA1c_ref = df.loc[df["项目名称"].isin(values)]
df = df.loc[df["项目名称"] !='糖化血红蛋白A1c',: ]
HbA1c_ref.head(2)

,,INSPECTION_ID,登记号,检查日期,检验目的,审核时间,项目名称,结果,单位,参考值,year,age,age_range
107,108.0,20160104G0460006,20422685,20160104,糖化血红蛋白,2016-01-04 12:16:12,糖化血红蛋白A1c,5.50,%,4.1-6.8,2016,29,"(22.0, 30.0]"
106,107.0,20160104G0460003,20544169,20160104,糖化血红蛋白,2016-01-04 12:15:53,糖化血红蛋白A1c,6.30,%,4.1-6.8,2016,26,"(22.0, 30.0]"


<b> 6）数据合规 </b>

检查日期字段取值是否合规，即是数值型字符串，将不符合规则的值作为缺失值处理，替换为“20000101”。

In [38]:
def is_number(num):
    pattern = re.compile(r'^[-+]?[-0-9]\d*\.\d*|[-+]?\.?[0-9]\d*$')
    result = pattern.match(num)
    if result:
        return True
    else:
        return False
date_na = 20000101
HbA1c_ref["检查日期"] = HbA1c_ref["检查日期"].apply(lambda x:np.where(is_number(str(x)),x,date_na))

<b>7) 数据匹配</b>

因同一体检者可能做过多次OGTT检测，在该案例中，我们假设HbA1c和最近一次的OGTT检测时，被检测者健康状态未发生改变，因此我们首先寻找做可作参考的OGTT的日期，记做字段OGTT_date。

In [39]:
HbA1c_ref["OGTT_date"] = 0   #给了日期字段一个默认值
#for index, row in HbA1c_ref.iterrows() :
def setvalue(row,df):
    id = row["登记号"]       #HbA1c_ref数据框的“登记号”字段：做HbA1c检测者的登记号
    date = row["检查日期"]    #HbA1c_ref数据框的登记号：做HbA1c检测者的登记号
    list1 = df.loc[(df["登记号"]== id) & ( df["检查日期"] < date) ,["检查日期"]]  #之前，最近一次做OGTT检测的日期
    if len(list1) :
        date1 = np.max(list1)[0]
    else:
        date1 =date_na        
    list2 = df.loc[(df["登记号"]== id) & ( df["检查日期"] >= date) ,["检查日期"]]  #之后，最近一次做OGTT检测的日期 
    if len(list2) :
        date2 = np.min(list2)[0]  
    else:
        date2 =date_na  
    diff1 = date-date1
    diff2 = date2-date  

    return np.where(np.min([diff1,diff2]) ==diff1,date1,date2)    # 最近一次，之前或之后
HbA1c_ref["OGTT_date"] = HbA1c_ref.apply(setvalue,axis=1,df = df) #填充每行的OGTT_date字段
HbA1c_ref = HbA1c_ref.loc[~HbA1c_ref["OGTT_date"].isna() & HbA1c_ref["OGTT_date"]!=date_na ]  #删除不能找到OGTT检测日期的记录
HbA1c_ref.head(2)

,,INSPECTION_ID,登记号,检查日期,检验目的,审核时间,项目名称,结果,单位,参考值,year,age,age_range,OGTT_date
107,108.0,20160104G0460006,20422685,20160104,糖化血红蛋白,2016-01-04 12:16:12,糖化血红蛋白A1c,5.50,%,4.1-6.8,2016,29,"(22.0, 30.0]",20160104
106,107.0,20160104G0460003,20544169,20160104,糖化血红蛋白,2016-01-04 12:15:53,糖化血红蛋白A1c,6.30,%,4.1-6.8,2016,26,"(22.0, 30.0]",20000101


<b>8. 数据填充 </b>

确定了OGTT_date的日期，接下来我们将利用每行的“登记号”和“OGTT_date”来填补OGTT的检查结果字段：'空腹血糖（妊娠）','75g一小时血糖(妊娠)','75g二小时血糖(妊娠)',方便后面分析。

In [40]:
def set_OGTT(row,df,type):
    id = row["登记号"]
    date = row["OGTT_date"]
    value =df.loc[(df["登记号"]== id) & ( df["检查日期"] ==date) & ( df["项目名称"]==type ) ,["结果"]] 
    try:
        value = value.iloc[0,0]
    except (Exception):
        value = None
        pass        
    return value
HbA1c_ref["OGTT_0"] = HbA1c_ref.apply(set_OGTT,axis=1,df = df,type = "空腹血糖（妊娠）")  
HbA1c_ref["OGTT_1"] = HbA1c_ref.apply(set_OGTT,axis=1,df = df,type = "75g一小时血糖(妊娠)")
HbA1c_ref["OGTT_2"] = HbA1c_ref.apply(set_OGTT,axis=1,df = df,type = "75g二小时血糖(妊娠)")

HbA1c_ref.head(2)

,,INSPECTION_ID,登记号,检查日期,检验目的,审核时间,项目名称,结果,单位,参考值,year,age,age_range,OGTT_date,OGTT_0,OGTT_1,OGTT_2
107,108.0,20160104G0460006,20422685,20160104,糖化血红蛋白,2016-01-04 12:16:12,糖化血红蛋白A1c,5.50,%,4.1-6.8,2016,29,"(22.0, 30.0]",20160104,4.90,8.40,7.40
106,107.0,20160104G0460003,20544169,20160104,糖化血红蛋白,2016-01-04 12:15:53,糖化血红蛋白A1c,6.30,%,4.1-6.8,2016,26,"(22.0, 30.0]",20000101,None,None,None


<b> 9. 数据过滤和转换 </b>

删除三个OGTT检测指标不完整的记录；

将“结果”字段重命名为“HbA1c”；

类型转换；

In [41]:
HbA1c_ref = HbA1c_ref.loc[~HbA1c_ref["OGTT_0"].isna() |~HbA1c_ref["OGTT_1"].isna() |~HbA1c_ref["OGTT_2"].isna()]  #去掉三个检测结果中有缺失值的行
HbA1c_ref.rename(columns={"结果":"HbA1c"},inplace = True)  
HbA1c_ref["HbA1c"] = HbA1c_ref["HbA1c"].astype("float")    
HbA1c_ref["OGTT_0"] = HbA1c_ref["OGTT_0"].astype("float")
HbA1c_ref["OGTT_1"] = HbA1c_ref["OGTT_1"].astype("float")
HbA1c_ref["OGTT_2"] = HbA1c_ref["OGTT_2"].astype("float")
#HbA1c_ref.shape[0]

<b> 10. 衍生字段 </b>

添加数据标签“STATUS”：根据OGTT检测的结果

In [42]:
HbA1c_ref["STATUS"] = "health"    # 定义一个字段“STATUS”,默认“health”健康，通过OGTT检测结果来判断是GDM还是DM
HbA1c_ref.loc[((~HbA1c_ref["OGTT_1"].isna()) & (HbA1c_ref["OGTT_1"]>=10) ) |((~HbA1c_ref["OGTT_2"].isna()) &  (HbA1c_ref["OGTT_2"]>=8.5)) | ((~HbA1c_ref["OGTT_0"].isna()) & (HbA1c_ref["OGTT_0"]>=5.1 )) ,"STATUS"] = "GDM"
HbA1c_ref.loc[((~HbA1c_ref["OGTT_2"].isna()) &  (HbA1c_ref["OGTT_2"]>=11.1)) | ((~HbA1c_ref["OGTT_0"].isna()) & (HbA1c_ref["OGTT_0"]>=7) ) , "STATUS"] = "DM"
HbA1c_ref.head(2)

,,INSPECTION_ID,登记号,检查日期,检验目的,审核时间,项目名称,HbA1c,单位,参考值,year,age,age_range,OGTT_date,OGTT_0,OGTT_1,OGTT_2,STATUS
107,108.0,20160104G0460006,20422685,20160104,糖化血红蛋白,2016-01-04 12:16:12,糖化血红蛋白A1c,5.5,%,4.1-6.8,2016,29,"(22.0, 30.0]",20160104,4.9,8.4,7.4,health
219,220.0,20160107G0460006,20185957,20160107,糖化血红蛋白,2016-01-07 14:43:57,糖化血红蛋白A1c,5.2,%,4.1-6.8,2016,36,"(35.0, 42.0]",20160107,4.8,7.4,5.3,health


# <b> 第四步  导出结果 </b>

非必要步骤

将中间过程报错为csv

In [43]:
HbA1c_ref.to_csv("../HbA1c检测在妊娠期糖尿病筛查的应用价值/output/clean_data.csv")  
print(" Successfully saved ！！ ")

 Successfully saved ！！ 
